In [1]:
import torch
from models.SamBlip import SamBlip
import numpy as np

/home/xcg/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SamBlip()
device = torch.device("cuda:1")
model.load_state_dict(torch.load("sam_blip_pretrained.pth",map_location = "cpu")) 
model = model.to(device)

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.6.intermediate_query.dense.bias', 'bert.encoder.layer.6.crossattention.self.query.bias', 'bert.encoder.layer.4.intermediate_query.dense.weight', 'bert.encoder.layer.1.intermediate_query.dense.weight', 'bert.encoder.layer.9.intermediate_query.dense.weight', 'bert.encoder.layer.11.output_query.LayerNorm.weight', 'bert.encoder.layer.8.output_query.LayerNorm.weight', 'bert.encoder.layer.10.intermediate_query.dense.weight', 'bert.encoder.layer.0.intermediate_query.dense.bias', 'bert.encoder.layer.9.output_query.LayerNorm.weight', 'bert.encoder.layer.5.output_query.LayerNorm.weight', 'bert.encoder.layer.4.output_query.LayerNorm.bias', 'bert.encoder.layer.2.crossattention.self.key.weight', 'bert.encoder.layer.10.output_query.LayerNorm.weight', 'bert.encoder.layer.5.output_query.dense.weight', 'bert.encoder.layer.11.intermediate_query.dense.bias', 

RuntimeError: Error(s) in loading state_dict for SamBlip:
	Missing key(s) in state_dict: "clip_qformer.cls_token", "sam_qformer.cls_token". 

# Load Paras


In [ ]:
import copy
path = "/data/xcg/lavis_data/output/BLIP2/best_results/checkpoint_2.pth"
checkpoint = torch.load(path, map_location = "cpu")
clip_state_dict = copy.deepcopy(checkpoint["model"])
sam_state_dict = copy.deepcopy(clip_state_dict)
tmp_state_dict = copy.deepcopy(clip_state_dict)
for key in list(sam_state_dict.keys()):
    if key.startswith("Qformer") or key.startswith("query_tokens"):
        del sam_state_dict[key]
    if "sam_" in key:
        temp = sam_state_dict[key]
        del sam_state_dict[key]
        sam_state_dict[key[4:]] = temp



model.clip_qformer.load_state_dict(clip_state_dict,strict=False)
model.sam_qformer.load_state_dict(sam_state_dict,strict=False)

for key in list(tmp_state_dict.keys()):
    if key.startswith("opt_proj"):
        tmp = tmp_state_dict[key]
        del tmp_state_dict[key]
        tmp_state_dict["clip_proj" + key[len("opt_proj"):]] = tmp
    elif key.startswith("sam_opt_proj"):
        tmp = tmp_state_dict[key]
        del tmp_state_dict[key]
        tmp_state_dict["sam_proj" + key[len("sam_opt_proj"):]] = tmp
    else:
        del tmp_state_dict[key]

model.load_state_dict(tmp_state_dict,strict=False)
torch.save(model.state_dict(), 'sam_blip_pretrained.pth')
# model.clip_proj.load_state_dict(tmp_state_dict)

# for key in tmp_state_dict.keys():
#     print(key)

In [ ]:
device = torch.device("cuda:1")
clip_features = np.load("/data2/xcg_data/lavis_data/2023us/features/clip_features/20230101_11548959_012117568.npz")['arr']
sam_features = np.load("/data2/xcg_data/lavis_data/2023us/features/sam_features/20230101_11548959_012117568.npz")['arr']
clip_features = torch.tensor(clip_features).to(device).unsqueeze(0)
sam_features = torch.tensor(sam_features).to(device).unsqueeze(0)

In [ ]:

output = model.forward(clip_features=clip_features, sam_features=sam_features, prompts=[""])
output

In [ ]:
output = model.generate(clip_features=clip_features, sam_features=sam_features, prompt="",max_length=100)
output

In [ ]:
torch.save(model.state_dict(), 'sam_blip_pretrained.pth')

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.size())

# Pure Playground

In [ ]:
class BaseClass():
    def __init__(self):
        pass

    def p(self):
        print("Base")

    def print(self):
        self.p()

class ExtClass(BaseClass):
    def __init__(self):
        super().__init__()
    
    def p(self):
        print("Ext")
    

In [ ]:
model = ExtClass()
model.print()